### Extraindo informações mais gerais dos produtos na categoria Mamãe e Bebê:
* Título
* Marca
* Preço com Desconto
* Preço sem desconto

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless") 


urls = [
    "https://drogariasoares.com.br/categoria/formulas-infantis/161",
    "https://drogariasoares.com.br/categoria/assaduras/101",
    "https://drogariasoares.com.br/categoria/fraldas/104",
    "https://drogariasoares.com.br/categoria/hora-do-banho/107",
    "https://drogariasoares.com.br/categoria/alimentos/100",
    "https://drogariasoares.com.br/categoria/bomba-de-tirar-leite/102",
    "https://drogariasoares.com.br/categoria/gestante/105",
    "https://drogariasoares.com.br/categoria/lenco-umedecido/108",
    "https://drogariasoares.com.br/categoria/anestesico-dental/134",
    "https://drogariasoares.com.br/categoria/colonias-e-hidratantes/103",
    "https://drogariasoares.com.br/categoria/higiene-bucal/106"
]


all_products_data = []


for url in urls:
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(3)  

    
    html_content = driver.page_source

   
    driver.quit()

    
    soup = BeautifulSoup(html_content, "html.parser")

    
    produtos = soup.find_all("div", class_="item-prod")

    
    for produto in produtos:
        titulo = produto.find("h2").text.strip()
        marca = produto.find("small", class_="marca").text.strip()
        preco_sem_desconto_element = produto.find("li", class_="preco text-center").find("p", class_='preco-de')
        preco_sem_desconto = preco_sem_desconto_element.text.strip() if preco_sem_desconto_element else ""
        preco_com_desconto = produto.find("p", class_="preco-por").strong.span.text.strip()

       
        product_data = {
            "Título": titulo,
            "Marca": marca,
            "Preço sem Desconto": preco_sem_desconto,
            "Preço com Desconto": preco_com_desconto
        }
        all_products_data.append(product_data)


df1 = pd.DataFrame(all_products_data)


df1.drop_duplicates(inplace=True)


C:\Users\lalaz\AppData\Local\Temp\ipykernel_72928\933582440.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto
0,LEITE NINHO FASES PREBIO 3+ 800G,ELK - ELK COMERCIO E REPRESENTACOES LTDA,,"R$ 53,99"
1,FORMULA INFANTIL LEITE APTAMIL 1 800G,DANONE,,"R$ 72,90"
2,LEITE NESTOGENO 1 800GR,NESTLE,,"R$ 59,90"
3,FORMULA INFANTIL APTAMIL PRO FUTURA 1 800G,DANONE,,"R$ 91,20"
4,FORMULA INFANTIL LEITE APTAMIL 2 800G**,DANONE,,"R$ 79,90"
...,...,...,...,...
124,GEL DENTAL INFANTIL GUM PATRULHA CANINA 50G,GUM,"R$ 7,90","R$ 6,90"
125,GEL DENTAL MALVATRIKIDS SABOR TUTTI-FRUTTI BAB...,MEGALABS FARMACEUTICA SA-OTC,,"R$ 21,20"
126,ESCOVA DENTAL ORAL-B KIDS MICKEY MACIA,PROCTER&GAMBLE-PERFUMARIA,,"R$ 8,29"
127,FIO DENTAL INFANTIL FLOSSER GUM TROLLS SABOR U...,GUM,,"R$ 14,90"


### Extraindo os códigos dos produtos na categoria Mamãe e Bebê:
* ID
* SKU
##### OBS: Infelizmente, no site da Drogaria Soares não foi localizado o código relativo ao EAN, por esse motivo, o mesmo não foi apresentado nesse código.

In [3]:
import concurrent.futures
import re  # Adicionada a importação do módulo re (Expressões Regulares)
import requests
from pyquery import PyQuery as pq
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

def extrair_links_produtos():
    urls = [

        "https://drogariasoares.com.br/categoria/formulas-infantis/161",
        "https://drogariasoares.com.br/categoria/assaduras/101",
        "https://drogariasoares.com.br/categoria/fraldas/104",
        "https://drogariasoares.com.br/categoria/hora-do-banho/107",
        "https://drogariasoares.com.br/categoria/alimentos/100",
        "https://drogariasoares.com.br/categoria/bomba-de-tirar-leite/102",
        "https://drogariasoares.com.br/categoria/gestante/105",
        "https://drogariasoares.com.br/categoria/lenco-umedecido/108",
        "https://drogariasoares.com.br/categoria/anestesico-dental/134",
        "https://drogariasoares.com.br/categoria/colonias-e-hidratantes/103",
        "https://drogariasoares.com.br/categoria/higiene-bucal/106"
    ]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
    links = []
    
    for url in urls:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            doc = pq(response.text)
            produtos = doc('.link-prod')
            
            for produto in produtos:
                href = pq(produto).attr('href')
                if 'a-4-body-lotion-maca-220g' not in href:
                    links.append(href)

    return links

def extrair_info_produto(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    time.sleep(5)

    page_source = driver.page_source

    doc = pq(page_source)
    
    script_content = doc('script').text()  # Obtém o conteúdo dos scripts na página

    # Usando expressão regular para encontrar o ID e o SKU nos scripts
    id_match = re.search(r"id:'(\d+)'", script_content)
    sku_match = re.search(r"sku:'(\d+)'", script_content)
    

    if id_match and sku_match:
        id_produto = id_match.group(1)
        sku_produto = sku_match.group(1)
        return id_produto, sku_produto
    else:
        return None, None

def extrair_info_produto_wrapper(url):
    try:
        return extrair_info_produto(url)
    except Exception as e:
        print(f"Erro ao extrair informações de {url}: {e}")
        return None, None

links_produtos = extrair_links_produtos()

dados_produtos = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    resultados = executor.map(extrair_info_produto_wrapper, ['https://drogariasoares.com.br' + link for link in links_produtos])
    for resultado in resultados:
        if resultado[0] is not None and resultado[1] is not None:
            dados_produtos.append(resultado)

df2 = pd.DataFrame(dados_produtos, columns=['ID do Produto', 'SKU'])
df2.drop_duplicates(inplace=True)




Erro ao extrair informações de https://drogariasoares.com.br/cetrilan-sab-100g-infantil/68551-01: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome-headless-shell=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00E28D03+51395]
	(No symbol) [0x00D95F61]
	(No symbol) [0x00C4E13A]
	(No symbol) [0x00C3ECFC]
	(No symbol) [0x00C3ECA3]
	(No symbol) [0x00C3E68E]
	(No symbol) [0x00C56D66]
	(No symbol) [0x00CBA87B]
	(No symbol) [0x00CA3C26]
	(No symbol) [0x00C7C629]
	(No symbol) [0x00C7D40D]
	GetHandleVerifier [0x011A68D3+3712147]
	GetHandleVerifier [0x011E5CBA+3971194]
	GetHandleVerifier [0x011E0FA8+3951464]
	GetHandleVerifier [0x00ED9D09+776393]
	(No symbol) [0x00DA1734]
	(No symbol) [0x00D9C618]
	(No symbol) [0x00D9C7C9]
	(No symbol) [0x00D8DDF0]
	BaseThreadInitThunk [0x76A07BA9+25]
	RtlInitializeExceptionChain [0x777FBDAB+107]
	RtlClearBits [0x777FBD2F+191]
Erro ao extrair

In [4]:
df2

,ID do Produto,SKU
0,9991011,103694
4,4973914,83919
8,4393933,83977
12,4973922,91094
16,4973916,83921
...,...,...
488,9990865,103544
492,1704801,84612
496,4839938,94383
500,9990864,103543


### Unificando e Tratando os Dataframes:

In [6]:
reindexed_df = df2.set_index(pd.Index(range(128)))

In [7]:
df1["ID"] = reindexed_df["ID do Produto"]
df1["SKU"] = reindexed_df["SKU"]

In [8]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto,ID,SKU
0,LEITE NINHO FASES PREBIO 3+ 800G,ELK - ELK COMERCIO E REPRESENTACOES LTDA,,"R$ 53,99",9991011,103694
1,FORMULA INFANTIL LEITE APTAMIL 1 800G,DANONE,,"R$ 72,90",4973914,83919
2,LEITE NESTOGENO 1 800GR,NESTLE,,"R$ 59,90",4393933,83977
3,FORMULA INFANTIL APTAMIL PRO FUTURA 1 800G,DANONE,,"R$ 91,20",4973922,91094
4,FORMULA INFANTIL LEITE APTAMIL 2 800G**,DANONE,,"R$ 79,90",4973916,83921
...,...,...,...,...,...,...
124,GEL DENTAL INFANTIL GUM PATRULHA CANINA 50G,GUM,"R$ 7,90","R$ 6,90",1704801,84612
125,GEL DENTAL MALVATRIKIDS SABOR TUTTI-FRUTTI BAB...,MEGALABS FARMACEUTICA SA-OTC,,"R$ 21,20",4839938,94383
126,ESCOVA DENTAL ORAL-B KIDS MICKEY MACIA,PROCTER&GAMBLE-PERFUMARIA,,"R$ 8,29",9990864,103543
127,FIO DENTAL INFANTIL FLOSSER GUM TROLLS SABOR U...,GUM,,"R$ 14,90",7889901,93408


In [9]:
df1.to_csv('Mamãe_e_Bebê_Soares.csv')